In [1]:
from osgeo import osr, ogr, gdal
import pyproj
from pyproj import Proj, transform
import geopandas as gpd
import numpy as np
import rasterio
import pandas as pd
import seaborn as sns
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
import math
import datetime

In [2]:
td = r"C:\Users\crowk\Downloads\td_600\td_coords_600.shp"
td_quant = 600

In [3]:
def getCoords(shp_path, fid):

    shapefile = ogr.Open(shp_path)
    layer = shapefile.GetLayer(0)
    
    #define projected->geographic point transformer for later use
    inProj = Proj('epsg:3857')
    outProj = Proj('epsg:4326')
    
    convert2Geo = False
    #conditional for whether or not transorming is necessary by determining if projected or geographic CRS
    if (gpd.read_file(shp_path).crs) == "epsg:3857":
        convert2Geo = True 
        
    feature = layer.GetFeature(fid)
    lon = feature['xcoord']
    lat = feature['ycoord']
    lc_class = feature['class']
    
        #if in projected CRS, transform extents to geographic
    if convert2Geo == True:
        finalLon,finalLat = transform(inProj,outProj,lon, lat)
        lon = finalLon
        lat = finalLat
    else:
        return lon,lat,lc_class
    return lon, lat, lc_class


In [4]:
UE_path = "C:/Users/crowk/Downloads/final_ccdc_export_ue.tif"
CC_path = "C:/Users/crowk/Downloads/final_ccdc_export_cc.tif"

ue = rasterio.open("C:/Users/crowk/Downloads/final_ccdc_export_ue.tif")
cc = rasterio.open("C:/Users/crowk/Downloads/final_ccdc_export_cc.tif")

UE_ds = gdal.Open(UE_path)
CC_ds = gdal.Open(CC_path)

UeArr = ue.read()
CcArr = cc.read()


In [5]:
def transform_4326_to_3857(lat, long):
    # make spatial references
    # 4326 in
    InSR = osr.SpatialReference()
    InSR.ImportFromEPSG(4326)
    # 3857 out
    OutSR = osr.SpatialReference()
    OutSR.ImportFromEPSG(3857)
    
    # create the point
    Point = ogr.Geometry(ogr.wkbPoint)
    Point.AddPoint(lat,long)

    # assign the spacial reference to the point
    Point.AssignSpatialReference(InSR)

    # transform
    transform_error_code = Point.TransformTo(OutSR)

    # error check (eg, point is out of bounds, etc)
    if transform_error_code == 0:
        # success
        return (Point.GetX(),Point.GetY())
    else:
        #failed
        logging.warn("Transform failed! Error {transform_error_code}")
        return None



In [6]:
def getArrIndex(x,y,raster_path_1, raster_path_2):
    # coordinates to get array values for
    points = [(x, y)]
    loc = 'UE'
    # open the raster file
    ds = gdal.Open(raster_path_1)
    boop = rasterio.open(raster_path_1).read()
    b,h,l = boop.shape

    # get georeference info
    transform = ds.GetGeoTransform() # (-2493045.0, 30.0, 0.0, 3310005.0, 0.0, -30.0)
    xOrigin = transform[0] # -2493045.0
    yOrigin = transform[3] # 3310005.0
    pixelWidth = transform[1] # 30.0
    pixelHeight = transform[5] # -30.0

    band = ds.GetRasterBand(1) # 1-based index
    data = band.ReadAsArray()
    # loop through the coordinates
    for point in points:
        x = point[0]
        y = point[1]

        xOffset = int((x - xOrigin) / pixelWidth)
        yOffset = int((y - yOrigin) / pixelHeight)
        # get individual pixel values
        if (0 < xOffset < l) is False or (0< yOffset < h) is False:
            #print('central city zone')
            ds = gdal.Open(raster_path_2)

            # get georeference info
            transform = ds.GetGeoTransform() # (-2493045.0, 30.0, 0.0, 3310005.0, 0.0, -30.0)
            xOrigin = transform[0] # -2493045.0
            yOrigin = transform[3] # 3310005.0
            pixelWidth = transform[1] # 30.0
            pixelHeight = transform[5] # -30.0

            band = ds.GetRasterBand(1) # 1-based index
            data = band.ReadAsArray()
            # loop through the coordinates
            for point in points:
                x = point[0]
                y = point[1]

                xOffset = int((x - xOrigin) / pixelWidth)
                yOffset = int((y - yOrigin) / pixelHeight)
                value = data[yOffset][xOffset]
                loc = 'CC'
            return xOffset,yOffset, loc
        else:
            #print('urbanizing edge zone')
            value = data[yOffset][xOffset]
    return xOffset,yOffset, loc


In [7]:
#use new location info to extract td 
#get coords of each td point
#convert coords to raster pixels 
#use pixels to index into array 
cl_arr = []
pd_list = []
#td_arr = pd.DataFrame()
shapefile = ogr.Open(td)
layer = shapefile.GetLayer(0)
for i in range(layer.GetFeatureCount()):
    lon, lat, lc_class = getCoords(td,i)
    feature = layer.GetFeature(i)
    cl_arr.append(lc_class)
    x,y = transform_4326_to_3857(lat,lon)
    X_ind, Y_ind, loc = getArrIndex(x,y,UE_path,CC_path)
   
    if (loc == 'UE'):
        new_col = pd.DataFrame(UeArr[0:60,Y_ind, X_ind])
        pd_list.append(new_col)
        
    else:
        new_col = pd.DataFrame(CcArr[0:60,Y_ind,X_ind])
        pd_list.append(new_col)
        
            
td_arr = pd.concat(pd_list,axis = 1, join = 'inner')


In [8]:
cl_arr = pd.DataFrame(np.array(cl_arr).reshape(1,td_quant))
td_arr = pd.DataFrame(np.array(td_arr))

td_list = pd.concat([cl_arr, td_arr], ignore_index=True).transpose()
td_list = td_list.drop(1, axis = 1)

In [9]:
b,h,l = UeArr.shape
print(b,h,l)

b2,h2,l2 = CcArr.shape
print(b2,h2,l2)

532 167 227
532 203 292


In [10]:
flat_UE = pd.DataFrame(UeArr.reshape(b,h*l))
flat_CC = pd.DataFrame(CcArr.reshape(b2,h2*l2))

flat_UE = flat_UE.drop(index =0).reset_index(drop=True)
flat_CC = flat_CC.drop(index =0).reset_index(drop=True)

In [11]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
display(flat_UE)


print(flat_UE.shape)
print(idx_UE.shape)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,37859,37860,37861,37862,37863,37864,37865,37866,37867,37868,37869,37870,37871,37872,37873,37874,37875,37876,37877,37878,37879,37880,37881,37882,37883,37884,37885,37886,37887,37888,37889,37890,37891,37892,37893,37894,37895,37896,37897,37898,37899,37900,37901,37902,37903,37904,37905,37906,37907,37908
0,6.288e+11,6.634e+11,6.288e+11,6.288e+11,6.634e+11,6.745e+11,7.076e+11,6.897e+11,6.897e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.634e+11,6.689e+11,6.634e+11,6.634e+11,6.288e+11,6.634e+11,6.634e+11,6.634e+11,6.288e+11,6.634e+11,6.662e+11,6.662e+11,6.634e+11,6.634e+11,6.634e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,...,6.634e+11,6.634e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.634e+11,6.634e+11,6.634e+11,6.634e+11,6.634e+11,6.288e+11,6.634e+11,6.634e+11,6.634e+11,6.689e+11,6.288e+11,6.288e+11,6.689e+11,6.634e+11,6.634e+11,6.634e+11,6.634e+11,6.634e+11,6.634e+11,6.662e+11,6.662e+11,6.662e+11,6.689e+11,6.689e+11,6.288e+11,6.288e+11,6.288e+11,6.634e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.288e+11,6.634e+11,6.634e+11
1,1.182e+12,8.763e+11,9.710e+11,9.599e+11,8.279e+11,1.212e+12,1.159e+12,9.447e+11,9.447e+11,1.649e+12,8.348e+11,7.657e+11,1.040e+12,1.040e+12,1.656e+12,1.650e+12,1.650e+12,1.212e+12,1.649e+12,1.650e+12,8.348e+11,1.065e+12,1.656e+12,1.656e+12,1.656e+12,1.656e+12,1.040e+12,1.003e+12,9.779e+11,9.779e+11,1.144e+12,9.813e+11,1.017e+12,1.017e+12,1.220e+12,1.359e+12,9.599e+11,1.656e+12,1.656e+12,1.445e+12,1.308e+12,1.308e+12,1.655e+12,1.652e+12,1.650e+12,1.212e+12,1.384e+12,1.212e+12,1.040e+12,1.040e+12,...,1.108e+12,1.024e+12,1.656e+12,1.323e+12,1.387e+12,1.387e+12,1.656e+12,1.082e+12,9.496e+11,9.247e+11,8.542e+11,9.247e+11,1.656e+12,1.656e+12,1.316e+12,1.656e+12,1.140e+12,1.656e+12,1.656e+12,1.655e+12,9.779e+11,1.068e+12,1.068e+12,9.406e+11,1.656e+12,1.656e+12,8.348e+11,9.572e+11,1.650e+12,1.268e+12,1.268e+12,1.519e+12,1.649e+12,1.656e+12,1.111e+12,1.111e+12,1.111e+12,1.285e+12,1.285e+12,1.655e+12,1.080e+12,1.656e+12,1.077e+12,1.656e+12,1.656e+12,1.382e+12,1.656e+12,1.656e+12,9.599e+11,1.246e+12
2,1.189e+12,8.887e+11,9.751e+11,9.655e+11,8.293e+11,1.215e+12,1.160e+12,9.496e+11,9.496e+11,1.650e+12,8.376e+11,7.685e+11,1.044e+12,1.044e+12,0.000e+00,1.652e+12,1.652e+12,1.212e+12,1.650e+12,1.652e+12,8.376e+11,1.068e+12,0.000e+00,0.000e+00,0.000e+00,0.000e+00,1.044e+12,1.012e+12,9.800e+11,9.800e+11,1.158e+12,9.862e+11,1.024e+12,1.024e+12,1.223e+12,1.361e+12,9.655e+11,0.000e+00,0.000e+00,1.446e+12,1.314e+12,1.314e+12,1.656e+12,1.653e+12,1.652e+12,1.212e+12,1.385e+12,1.212e+12,1.044e+12,1.044e+12,...,1.111e+12,1.035e+12,0.000e+00,1.325e+12,1.388e+12,1.388e+12,0.000e+00,1.086e+12,9.509e+11,9.378e+11,8.708e+11,9.378e+11,0.000e+00,0.000e+00,1.317e+12,0.000e+00,1.156e+12,0.000e+00,0.000e+00,1.656e+12,9.800e+11,1.069e+12,1.069e+12,9.447e+11,0.000e+00,0.000e+00,8.376e+11,9.578e+11,1.652e+12,1.270e+12,1.270e+12,1.520e+12,1.650e+12,0.000e+00,1.112e+12,1.112e+12,1.112e+12,1.286e+12,1.286e+12,1.656e+12,1.082e+12,0.000e+00,1.080e+12,0.000e+00,0.000e+00,1.385e+12,0.000e+00,0.000e+00,9.655e+11,1.250e+12
3,1.137e-01,2.768e-01,1.349e-01,1.010e-01,2.589e-01,1.606e-01,2.948e-01,3.335e-01,3.335e-01,1.077e-01,7.777e-02,1.061e-02,5.460e-02,1.026e-01,9.584e-02,8.706e-02,8.706e-02,9.732e-02,8.376e-02,8.582e-02,-3.069e-02,5.949e-02,8.721e-02,9.721e-02,9.721e-02,9.977e-02,9.969e-02,1.070e-01,-1.903e-03,1.485e-02,1.142e-01,1.178e-01,1.180e-01,1.180e-01,1.078e-01,3.718e-02,5.907e-02,1.105e-01,1.029e-01,9.192e-02,9.224e-02,9.224e-02,9.441e-02,9.683e-02,1.021e-01,1.079e-01,1.035e-01,1.006e-01,9.942e-02,9.942e-02,...,8.959e-02,9.045e-02,9.097e-02

(531, 37909)


NameError: name 'idx_UE' is not defined

In [12]:
#prepare class label df!
X = np.array(td_list.iloc[:,4:]) #train model on coefficients only
y = td_list.iloc[:,0] #class labels
print(y.shape)
np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)
print(X_train.shape)

(600,)
(480, 56)


In [13]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
#increase training samples, try getting training accuracy of 85%
clf = RandomForestClassifier(max_depth=4, random_state=0, n_estimators = 1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred)
y_pred_2 = clf.predict(X_train)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_2))

[3. 1. 3. 3. 3. 3. 3. 5. 3. 3. 2. 3. 4. 3. 3. 3. 3. 3. 3. 4. 5. 3. 3. 3.
 4. 3. 3. 3. 2. 4. 3. 3. 5. 3. 3. 2. 5. 3. 3. 5. 5. 4. 2. 3. 2. 4. 5. 2.
 1. 1. 5. 3. 4. 3. 1. 3. 3. 2. 3. 5. 5. 3. 3. 3. 5. 3. 3. 3. 2. 3. 4. 2.
 2. 2. 2. 3. 4. 3. 3. 2. 5. 3. 3. 3. 4. 5. 3. 3. 4. 2. 3. 3. 3. 2. 3. 3.
 5. 5. 3. 3. 3. 5. 5. 3. 3. 4. 3. 3. 3. 4. 5. 3. 2. 2. 3. 3. 4. 3. 3. 3.]
Accuracy: 0.7
Accuracy: 0.8375


In [ ]:
ue_year_arr = np.zeros([30,1,h*l])

In [ ]:

for col in flat_UE[0:]:
    row_count = 0
    prev_year = 0
    yr_count = 1
    done = False
    for row in range(530):
        if row % 59 ==0:
            if math.isnan(flat_UE.loc[row,col]) != False and done !=True:
                #print(row,col, 'no more breaks!')
                ue_year_arr[prev_year:,0,col] = class_pred
                prev_year = 0
                row_count = 0
                yr_count = 1
                done = True 
            elif math.isnan(flat_UE.loc[row,col]) != True:
                #print(row,col, 'new seg')
                tStart = flat_UE.loc[row,col]
                #print(tStart)  
                tEnd = flat_UE.loc[row+1,col]
                #print(tEnd)
                range1 = (tEnd/1000)-(tStart/1000)
                num_years = range1/(31556926)
                if num_years > 0.5:
                    num_years = round(num_years)
                    yr_count = yr_count + num_years
                    class_pred = clf.predict(np.array(flat_UE.loc[row+3:row+58, col]).reshape(1, -1))
                    ue_year_arr[prev_year:prev_year+num_years:,0,col] = class_pred
                    prev_year = prev_year + num_years
                   
                else:
                    k = 0
                    #print('less than one year')
                               
print(done)


In [ ]:
ue_year_arr = ue_year_arr.reshape(30, h,l)

In [ ]:
def write_geotiff(filename, arr, in_ds):
    if arr.dtype == np.float32:
        arr_type = gdal.GDT_Float32
    else:
        arr_type = gdal.GDT_Int32

    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(filename, arr.shape[2], arr.shape[1], arr.shape[0], arr_type)
    print(arr.shape[2],arr.shape[1],arr.shape[0])
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    for i, image in enumerate(arr, 1):
        out_ds.GetRasterBand(i).WriteArray(image)
        out_ds.GetRasterBand(i)
    out_ds.FlushCache()
 

In [ ]:
write_geotiff("ue_ccdc.tif", ue_year_arr, UE_ds)   

In [ ]:
cc_year_arr = np.zeros([30,1,h2*l2])

In [ ]:

for col in flat_CC[0:]:
    row_count = 0
    prev_year = 0
    yr_count = 1
    done = False
    for row in range(530):
        if row % 59 ==0:
            if math.isnan(flat_CC.loc[row,col]) != False and done !=True:
                #print(row,col, 'no more breaks!')
                cc_year_arr[prev_year:,0,col] = class_pred
                prev_year = 0
                row_count = 0
                yr_count = 1
                done = True 
            elif math.isnan(flat_CC.loc[row,col]) != True:
                #print(row,col, 'new seg')
                tStart = flat_CC.loc[row,col]
                #print(tStart)  
                tEnd = flat_CC.loc[row+1,col]
                #print(tEnd)
                range1 = (tEnd/1000)-(tStart/1000)
                num_years = range1/(31556926)
                if num_years > 0.5:
                    num_years = round(num_years)
                    yr_count = yr_count + num_years
                    class_pred = clf.predict(np.array(flat_CC.loc[row+3:row+58, col]).reshape(1, -1))
                    cc_year_arr[prev_year:prev_year+num_years:,0,col] = class_pred
                    prev_year = prev_year + num_years
                    
                else:
                    k = 0
                    #print('less than one year')
                               
print('done')

In [ ]:
cc_year_arr = cc_year_arr.reshape(30, h2,l2)

In [ ]:
write_geotiff("cc_ccdc.tif", cc_year_arr, CC_ds)